<a href="https://colab.research.google.com/github/will-cotton4/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/module2-join-datasets/LS_DS_122_Join_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Join datasets

Objectives
- concatenate data with pandas
- merge data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Combine Data Sets: Standard Joins
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join

## Download data

We’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-03-26 16:02:31--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.227.235
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.227.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  43.3MB/s    in 4.9s    

2019-03-26 16:02:37 (40.1 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [1]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [5]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


## Goal: Reproduce this example

The first two orders for user id 1:

In [4]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

## Load data

Here's a list of all six CSV filenames

In [6]:
!ls -lh

total 681M
-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


For each CSV
- Load it with pandas
- Look at the dataframe's shape
- Look at its head (first rows)
- `display(example)`
- Which columns does it have in common with the example we want to reproduce?

### aisles

In [9]:
import pandas as pd
aisles = pd.read_csv('aisles.csv')
aisles.shape

(134, 2)

In [8]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [10]:
display(example)

Doesn't have anything we need

### departments

In [0]:
departments = pd.read_csv('departments.csv')

In [14]:
departments.shape

(21, 2)

In [12]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


Also don't need anything from here.

### order_products__prior

In [0]:
import pandas as pd
order_products_prior = pd.read_csv('order_products__prior.csv')

In [3]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [17]:
display(example)

We do have some stuff in here that we could use, looks like:

####order_id, product_id, and add_to_cart_order

### order_products__train

In [4]:
order_products_train = pd.read_csv('order_products__train.csv')
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


We need:
####order_id, product_id, add_to_cart_order

### orders

In [5]:
orders = pd.read_csv('orders.csv')
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


We need:
#### order_id, user_id, order_number, order_dow, order_hour_of_day

### products

In [6]:
products = pd.read_csv('products.csv')
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


We need:

#### product_id, product_name

## Concatenate order_products__prior and order_products__train

In [7]:
order_products = pd.concat([order_products_prior, order_products_train])
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## Get a subset of orders — the first two orders for user id 1

In [0]:
condition = (orders['user_id'] == 1) & (orders['order_number'] <= 2)

columns = ['user_id', 
           'order_id', 
           'order_number', 
           'order_dow', 
           'order_hour_of_day']

subset = orders.loc[condition, columns]

## Merge dataframes

Merge subset from `orders` with columns from `order_products`

In [29]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [30]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


Note: Can't just smash them together; the structures don't match, so we can't concatenate directly.

In [35]:
orders_merged = pd.merge(subset, 
         order_products[['order_id', 'add_to_cart_order', 'product_id']])
orders_merged.head()

,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id
0,1,2539329,1,2,8,1,196
1,1,2539329,1,2,8,2,14084
2,1,2539329,1,2,8,3,12427
3,1,2539329,1,2,8,4,26088
4,1,2539329,1,2,8,5,26405


In [36]:
display(example)

Merge with columns from `products`

In [54]:
final_test = pd.merge(orders_merged, products[['product_id', 'product_name']])
final_test.sort_values(['order_number','add_to_cart_order']).head(11)

,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id,product_name
0,1,2539329,1,2,8,1,196,Soda
2,1,2539329,1,2,8,2,14084,Organic Unsweetened Vanilla Almond Milk
3,1,2539329,1,2,8,3,12427,Original Beef Jerky
5,1,2539329,1,2,8,4,26088,Aged White Cheddar Popcorn
7,1,2539329,1,2,8,5,26405,XL Pick-A-Size Paper Towel Rolls
1,1,2398795,2,3,7,1,196,Soda
8,1,2398795,2,3,7,2,10258,Pistachios
4,1,2398795,2,3,7,3,12427,Original Beef Jerky
9,1,2398795,2,3,7,4,13176,Bag of Organic Bananas
6,1,2398795,2,3,7,5,26088,Aged White Cheddar Popcorn


# Assignment

These are the top 10 most frequently ordered products. **How many times was each ordered? **

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk


**Because the data we're dealing with here is so large that it's crashed my runtime a few times, I'm going to re-save the CSV's as DataFrames down here:**

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

%cd instacart_2017_05_01

--2019-03-26 19:45:40--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.176.61
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.176.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  17.7MB/s    in 15s     

2019-03-26 19:45:55 (13.3 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_0

In [0]:
import pandas as pd

order_products = pd.concat([pd.read_csv('order_products__prior.csv'), pd.read_csv('order_products__train.csv')])
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')


**First, write down which columns you need and which dataframes have them.**


**Need the following:**
- `order_id`
- `product_id`

**We can find these in a merged dataframe from `order_products` and `products`**

In [3]:
order_products.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [4]:
products.head(10)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7



**Next, merge these into a single dataframe.**


In [5]:
product_data_merged = pd.merge(order_products, products)
product_data_merged.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16



**Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.**


In [6]:
product_data_merged['product_name'].value_counts().head(10)

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64


## Stretch challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

**Your challenge is to reproduce the list of the top 25 latest ordered popular products.**

We'll define "popular products" as products with more than 2,900 orders.

**We need to include the hour of day in our dataframe, so I'll merge the `order_products`, `orders`, and `products` dataframes:**

In [4]:
complete_orders = pd.merge(pd.merge(order_products, orders), products)
complete_orders.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,26,33120,5,0,153404,prior,2,0,16,7.0,Organic Egg Whites,86,16
2,120,33120,13,0,23750,prior,11,6,8,10.0,Organic Egg Whites,86,16
3,327,33120,5,1,58707,prior,21,6,9,8.0,Organic Egg Whites,86,16
4,390,33120,28,1,166654,prior,48,0,12,9.0,Organic Egg Whites,86,16


**Here we split off the counts for purchases made early in the day:**

In [0]:
early_purchase_counts = complete_orders[ ( complete_orders['order_hour_of_day'] == 8 ) | ( complete_orders['order_hour_of_day'] == 9 ) ]['product_name'].value_counts()
                        



**And divide the result component-wise, so that the resulting `Series` records the proportions of the desired objects purchased in the desired timeframe: **

In [0]:
early_purchase_props = early_purchase_counts.divide(complete_orders['product_name'].value_counts().where(complete_orders['product_name'].value_counts()>=2900)).sort_values(ascending=False)

**Here they are: different from the ones on the graph, for some reason...**

In [128]:
print(early_purchase_props[:25])

Half And Half Ultra Pasteurized            0.160267
Trail Mix                                  0.147607
Pub Mix                                    0.146908
Nutri Grain Bars Multi Pack                0.144054
Apples                                     0.141815
Milk Chocolate Almonds                     0.140584
Sweet & Salty Nut Granola Bars Peanut      0.140157
0% Greek Strained Yogurt                   0.137499
Cheez-It Cheddar Cracker                   0.137435
Popcorn                                    0.136440
French Vanilla Coffee Creamer              0.136247
Half And Half Cream                        0.135763
Dry Roasted Almonds                        0.135509
Extra Fancy Unsalted Mixed Nuts            0.135394
Crunchy Oats 'n Honey Granola Bars         0.133891
Blackberry Yogurt                          0.133611
Original Rice Krispies Treats              0.132825
Cinnamon Brown Sugar Breakfast Biscuits    0.132770
with Crispy Almonds Cereal                 0.129535
Cheez-It Bak

**And then we do the same thing for the late purchases:**

In [0]:
late_purchase_counts =  complete_orders[ ( complete_orders['order_hour_of_day'] == 16) |
                                         ( complete_orders['order_hour_of_day'] == 17) ]['product_name'].value_counts()

In [0]:
late_purchase_props = late_purchase_counts.divide(complete_orders['product_name'].value_counts().where(complete_orders['product_name'].value_counts()>=2900)).sort_values(ascending=False)

In [129]:
print(late_purchase_props[:25])

Cabernet Sauvignon                           0.204188
Half Baked® Ice Cream                        0.192864
Rising Crust Pepperoni Pizza                 0.192596
Chardonnay                                   0.190356
Milk and Cookies Ice Cream                   0.188397
Sauvignon Blanc                              0.188151
Americone Dream® Ice Cream                   0.188150
The Tonight Dough™ Ice Cream                 0.188078
Phish Food® Ice Cream                        0.187913
Four Cheese Rising Crust Pizza               0.187800
Sicilian Pistachio                           0.187540
Pinot Grigio                                 0.187407
Cherry Garcia Ice Cream                      0.187097
Coconut Almond Minis Frozen Dessert Bars     0.184393
Pinot Noir                                   0.184159
Mint Chocolate Chip Ice Cream                0.182225
Coffee Ice Cream                             0.182187
Rice Mac & Cheese                            0.182003
Chocolate Peanut Butter Cup 